## Bibliotecas

In [0]:
import pyspark.sql.functions as F
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## Importando dataframe

In [0]:
%sql

SELECT * FROM telecom.silver.teleco_cleaned
LIMIT 10;

In [0]:
# Utilizando Query do SQL para salvar em um dataframe Spark
df = spark.sql('SELECT * FROM telecom.silver.teleco_cleaned')
display(df.head(10))